<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
%pip install beautifulsoup4
%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [7]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [8]:
# use requests.get() method with the provided static_url
# assign the response to a object

import requests

# URL of the snapshot (replace with the actual static URL for June 9, 2021)
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1023456789"

# Perform GET request and assign response to an object
response = requests.get(static_url)
# Check if request was successful
if response.status_code == 200:
    print("Request successful!")
else:
    print("Request failed with status code:", response.status_code)

Request successful!


In [9]:
import requests
import pandas as pd

# GET request to SpaceX API for past launches
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)
data = response.json()

# Convert JSON data to a flat DataFrame
df = pd.json_normalize(data)

# Convert the static_fire_date_utc column to datetime
df['static_fire_date_utc'] = pd.to_datetime(df['static_fire_date_utc'], errors='coerce')

# Extract the year from the first row
first_year = df['static_fire_date_utc'].iloc[0].year
print("Year of first row:", first_year)

Year of first row: 2006


Create a `BeautifulSoup` object from the HTML `response`


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

# Static Wikipedia snapshot
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Step 1: Request page
response = requests.get(static_url)
html_content = response.text

# Step 2: Parse with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Step 3: Find all tables with class 'wikitable'
tables = soup.find_all('table', {'class': 'wikitable'})

print(f"Found {len(tables)} tables.")

# Step 4: Convert the first table to a dataframe
table_html = str(tables[0])
df = pd.read_html(StringIO(table_html))[0]

print(df.head())

Found 13 tables.
   Flight No.                                Date and time (UTC)  \
0           1                                 4 June 2010, 18:45   
1           1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2           2                         8 December 2010, 15:43[13]   
3           2  Maiden flight of Dragon capsule, consisting of...   
4           3                             22 May 2012, 07:44[17]   

                                Version, Booster [b]  \
0                              F9 v1.0[7] B0003.1[8]   
1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2                              F9 v1.0[7] B0004.1[8]   
3  Maiden flight of Dragon capsule, consisting of...   
4                              F9 v1.0[7] B0005.1[8]   

                                         Launch site  \
0                                      CCAFS, SLC-40   
1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2                                      CCAFS, SLC-40   
3  Maiden fli

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [15]:
# Check if the BeautifulSoup object was created correctly
if soup.title:
    print("Page title:", soup.title.string)
else:
    print("No title tag found in the page.")

Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [14]:
import requests
import pandas as pd

# Fetch launches
launches_url = "https://api.spacexdata.com/v4/launches/past"
launches_data = requests.get(launches_url).json()
launches_df = pd.DataFrame(launches_data)

# Fetch rockets
rockets_url = "https://api.spacexdata.com/v4/rockets"
rockets_data = requests.get(rockets_url).json()
rockets_df = pd.DataFrame(rockets_data)

# Find Falcon 9 rocket ID
falcon9_id = rockets_df[rockets_df['name'] == 'Falcon 9']['id'].values[0]

# Filter launches that used Falcon 9
falcon9_launches = launches_df[launches_df['rocket'] == falcon9_id]

# Count
print("Number of Falcon 9 launches:", len(falcon9_launches))


Number of Falcon 9 launches: 179


In [11]:
import requests
import pandas as pd

# Fetch past launches
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)
data = response.json()

# Flatten the 'cores' nested list
cores_df = pd.json_normalize(data, record_path=['cores'])

# Check column names to be sure
print(cores_df.columns)

Index(['core', 'flight', 'gridfins', 'legs', 'reused', 'landing_attempt',
       'landing_success', 'landing_type', 'landpad'],
      dtype='object')


In [8]:
# Count missing values in 'landpad'
missing_landingpad = cores_df['landpad'].isnull().sum()
print("Number of missing values in landingPad column:", missing_landingpad)

Number of missing values in landingPad column: 36


In [10]:
# See how many entries are actually null
missing_landingpad = cores_df['landpad'].isnull().sum()
total_cores = len(cores_df)

print(f"Total cores: {total_cores}")
print(f"Missing landpad values: {missing_landingpad}")

Total cores: 193
Missing landpad values: 36


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [16]:
# Find all tables in the page
html_tables = soup.find_all('table')

# Check how many tables were found
print(f"Found {len(html_tables)} tables.")

Found 25 tables.


Starting from the third table is our target table contains the actual launch records.


In [17]:
# Print the third table to inspect its content (index 2 since Python is 0-based)
target_table = html_tables[2]
print(target_table.prettify()[:1000])  # print first 1000 characters for readability

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [7]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [8]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [18]:
import pandas as pd

# Step 1: Extract column names from the table header
headers = []
for th in target_table.find_all('th'):
    headers.append(th.text.strip())

# Step 2: Create an empty dictionary with column names as keys
launch_dict = {header: [] for header in headers}

# Step 3: Convert the dictionary to a Pandas DataFrame
launch_df = pd.DataFrame(launch_dict)

# Step 4: Display the empty DataFrame with column names
print(launch_df.head())


Empty DataFrame
Columns: [Flight No., Date andtime (UTC), Version,Booster [b], Launch site, Payload[c], Payload mass, Orbit, Customer, Launchoutcome, Boosterlanding, 1, 2, 3, 4, 5, 6, 7]
Index: []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
